# Session 10: KNN and Unsupervised Machine Learning

## Overview
This session covers both K-Nearest Neighbors (KNN) - a supervised learning algorithm, and Unsupervised Machine Learning focusing on K-Means clustering. We'll explore how both algorithms use the concept of "k" but for different purposes.

### Topics Covered:
1. **K-Nearest Neighbors (KNN) Algorithm**
   - Classification and regression
   - Choosing optimal k
   - Distance metrics and scaling
2. **Introduction to Unsupervised Learning**
3. **K-Means Clustering Algorithm**
4. **Determining Optimal Number of Clusters (K)**
   - Elbow Method
   - Silhouette Analysis
5. **Comparison: KNN vs K-Means**
6. **Practical Implementation and Evaluation**

### Learning Objectives:
- Understand KNN algorithm for supervised learning
- Learn principles of unsupervised learning
- Master K-Means clustering algorithm
- Compare and contrast KNN and K-Means
- Apply optimal k selection techniques for both algorithms
- Evaluate model performance and clustering quality

## K-Nearest Neighbors (KNN) Algorithm

### What is KNN?
K-Nearest Neighbors is a **supervised learning** algorithm that can be used for both classification and regression. It's called "lazy learning" because it doesn't build an explicit model during training - instead, it stores all training data and makes predictions based on the k nearest neighbors.

### How KNN Works:
1. **Store** all training data points
2. **Calculate distance** from new point to all training points
3. **Find k nearest** neighbors
4. **Make prediction**:
   - **Classification**: Majority vote of k neighbors
   - **Regression**: Average of k neighbors' values

### Key Concepts:
- **k**: Number of neighbors to consider
- **Distance metric**: Usually Euclidean distance
- **Voting**: How neighbors influence the prediction
- **Curse of dimensionality**: Performance degrades in high dimensions

### Applications:
- Recommendation systems
- Pattern recognition
- Image classification
- Text mining
- Market research

### KNN Classification Example

In [ ]:
# Create a synthetic classification dataset
X_class, y_class = make_classification(n_samples=300, n_features=2, n_redundant=0, 
                                      n_informative=2, n_clusters_per_class=1, 
                                      random_state=42)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, 
                                                    test_size=0.3, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Number of classes: {len(np.unique(y_class))}")

# Visualize the dataset
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_class[:, 0], X_class[:, 1], c=y_class, cmap='viridis', alpha=0.7)
plt.title('Synthetic Classification Dataset')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(scatter)
plt.show()

In [ ]:
# Test different values of k
k_values = [1, 3, 5, 7, 9, 15]
accuracies = []

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, k in enumerate(k_values):
    # Create and train KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Make predictions
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    
    # Create decision boundary visualization
    h = 0.02  # step size in the mesh
    x_min, x_max = X_class[:, 0].min() - 1, X_class[:, 0].max() + 1
    y_min, y_max = X_class[:, 1].min() - 1, X_class[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # Predict on mesh points
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot decision boundary
    axes[i].contourf(xx, yy, Z, alpha=0.4, cmap='viridis')
    scatter = axes[i].scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap='viridis', 
                             edgecolors='black')
    axes[i].set_title(f'KNN (k={k}), Accuracy: {accuracy:.3f}')
    axes[i].set_xlabel('Feature 1')
    axes[i].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

# Plot accuracy vs k
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.title('KNN Classification: Accuracy vs k')
plt.grid(True, alpha=0.3)
best_k = k_values[np.argmax(accuracies)]
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, 
           label=f'Best k = {best_k}')
plt.legend()
plt.show()

print(f"Best k value: {best_k} with accuracy: {max(accuracies):.3f}")

### KNN on Iris Dataset

In [ ]:
# Load Iris dataset
iris = load_iris()
X_iris_full = iris.data
y_iris_full = iris.target

# Split the data
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris_full, y_iris_full, test_size=0.3, random_state=42, stratify=y_iris_full
)

# Scale the features (important for KNN)
scaler = StandardScaler()
X_iris_train_scaled = scaler.fit_transform(X_iris_train)
X_iris_test_scaled = scaler.transform(X_iris_test)

# Test different k values for Iris
k_range = range(1, 16)
iris_accuracies = []

for k in k_range:
    knn_iris = KNeighborsClassifier(n_neighbors=k)
    knn_iris.fit(X_iris_train_scaled, y_iris_train)
    y_iris_pred = knn_iris.predict(X_iris_test_scaled)
    accuracy = accuracy_score(y_iris_test, y_iris_pred)
    iris_accuracies.append(accuracy)

# Plot results
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(k_range, iris_accuracies, 'go-', linewidth=2, markersize=6)
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Accuracy')
plt.title('KNN on Iris Dataset: Accuracy vs k')
plt.grid(True, alpha=0.3)
best_k_iris = k_range[np.argmax(iris_accuracies)]
plt.axvline(x=best_k_iris, color='red', linestyle='--', alpha=0.7)

# Use best k for final model
knn_best = KNeighborsClassifier(n_neighbors=best_k_iris)
knn_best.fit(X_iris_train_scaled, y_iris_train)
y_iris_final_pred = knn_best.predict(X_iris_test_scaled)

# Confusion matrix
plt.subplot(1, 2, 2)
cm = confusion_matrix(y_iris_test, y_iris_final_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Confusion Matrix (k={best_k_iris})')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()

print(f"Best k for Iris: {best_k_iris}")
print(f"Best accuracy: {max(iris_accuracies):.3f}")
print(f"\\nClassification Report:")
print(classification_report(y_iris_test, y_iris_final_pred, target_names=iris.target_names))

### Choosing Optimal k in KNN

#### Key Considerations:
1. **Odd vs Even k**: Use odd k for binary classification to avoid ties
2. **Small k**: More sensitive to local patterns (high variance, low bias)
3. **Large k**: More stable but may lose local patterns (low variance, high bias)
4. **Rule of thumb**: k = √n (where n is number of training samples)

#### Cross-Validation Approach:
Use cross-validation to find optimal k by testing different values and choosing the one with best average performance.

### KNN vs K-Means: Key Differences

| Aspect | KNN | K-Means |
|--------|-----|---------|
| **Learning Type** | Supervised | Unsupervised |
| **Purpose** | Classification/Regression | Clustering |
| **Training** | Lazy (no training phase) | Eager (builds model) |
| **Prediction** | Based on k nearest neighbors | Assigns to nearest centroid |
| **k meaning** | Number of neighbors | Number of clusters |
| **Distance** | To find neighbors | To assign clusters |
| **Output** | Class label or value | Cluster assignment |

### Important Notes:
- **Feature Scaling**: Always scale features for KNN (distance-based)
- **Curse of Dimensionality**: Performance degrades in high dimensions
- **Computational Cost**: KNN is expensive at prediction time
- **Memory Usage**: Stores all training data

## 1. Introduction to Unsupervised Learning

### What is Unsupervised Learning?
Unsupervised learning algorithms find patterns in data without being given explicit target labels. Main types include:

1. **Clustering**: Grouping similar data points together
2. **Dimensionality Reduction**: Reducing the number of features while preserving information
3. **Association Rules**: Finding relationships between different variables
4. **Anomaly Detection**: Identifying unusual data points

### Applications:
- Customer segmentation
- Market research
- Image segmentation
- Gene sequencing
- Recommendation systems

In [1]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, load_iris, make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Load and Prepare Sample Dataset

In [ ]:
# Create a synthetic dataset with known clusters for demonstration
X_synthetic, y_true = make_blobs(n_samples=300, centers=4, n_features=2, 
                                random_state=42, cluster_std=1.5)

print(f"Synthetic dataset shape: {X_synthetic.shape}")
print(f"Number of true clusters: {len(np.unique(y_true))}")

# Also load the Iris dataset for real-world example
iris = load_iris()
X_iris = iris.data[:, :2]  # Use only first 2 features for visualization
y_iris_true = iris.target

print(f"\nIris dataset shape: {X_iris.shape}")
print(f"Iris features used: {iris.feature_names[:2]}")
print(f"Number of true species: {len(np.unique(y_iris_true))}")

## 3. Visualize the Data

In [ ]:
# Visualize both datasets
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Synthetic data
scatter1 = axes[0].scatter(X_synthetic[:, 0], X_synthetic[:, 1], c=y_true, 
                          cmap='viridis', alpha=0.7)
axes[0].set_title('Synthetic Dataset (True Clusters)')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
plt.colorbar(scatter1, ax=axes[0])

# Iris data
scatter2 = axes[1].scatter(X_iris[:, 0], X_iris[:, 1], c=y_iris_true, 
                          cmap='viridis', alpha=0.7)
axes[1].set_title('Iris Dataset (True Species)')
axes[1].set_xlabel('Sepal Length (cm)')
axes[1].set_ylabel('Sepal Width (cm)')
plt.colorbar(scatter2, ax=axes[1])

plt.tight_layout()
plt.show()

## 4. K-Means Clustering Algorithm

### How K-Means Works:
1. **Initialize**: Choose K cluster centers randomly
2. **Assign**: Assign each point to the nearest cluster center
3. **Update**: Move cluster centers to the mean of assigned points
4. **Repeat**: Steps 2-3 until convergence

### Key Parameters:
- **n_clusters (k)**: Number of clusters
- **init**: Method for initialization ('k-means++' is default)
- **max_iter**: Maximum number of iterations
- **random_state**: For reproducible results

In [ ]:
# Implement basic K-Means clustering with a fixed K
def apply_kmeans(X, k, title):
    """Apply K-Means clustering and visualize results"""
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(X)
    centroids = kmeans.cluster_centers_
    
    # Calculate metrics
    inertia = kmeans.inertia_  # Within-cluster sum of squares
    silhouette_avg = silhouette_score(X, cluster_labels)
    
    # Visualization
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(X[:, 0], X[:, 1], c=cluster_labels, cmap='viridis', alpha=0.7)
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    plt.title(f'{title} - K={k}\nInertia: {inertia:.2f}, Silhouette Score: {silhouette_avg:.3f}')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.colorbar(scatter)
    plt.show()
    
    return kmeans, inertia, silhouette_avg

# Apply K-Means with K=4 to synthetic data
print("K-Means Clustering on Synthetic Data:")
kmeans_synthetic, _, _ = apply_kmeans(X_synthetic, 4, "Synthetic Dataset")

In [ ]:
# Apply K-Means with K=3 to Iris data
print("K-Means Clustering on Iris Data:")
kmeans_iris, _, _ = apply_kmeans(X_iris, 3, "Iris Dataset")

## 5. Elbow Method for Optimal K

### What is the Elbow Method?
The elbow method plots the Within-Cluster Sum of Squares (WCSS) against different values of K. The "elbow" point where the rate of decrease sharply changes indicates the optimal K.

### WCSS (Inertia):
Sum of squared distances from each point to its cluster centroid. Lower values indicate tighter clusters.

In [ ]:
def elbow_method(X, max_k=10, title="Elbow Method"):
    """Apply elbow method to find optimal K"""
    k_range = range(1, max_k + 1)
    inertias = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    
    # Plot the elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
    plt.title(f'{title} - Elbow Method')
    plt.grid(True, alpha=0.3)
    
    # Calculate and plot the rate of change
    if len(inertias) > 2:
        differences = np.diff(inertias)
        second_differences = np.diff(differences)
        elbow_point = np.argmax(second_differences) + 2  # +2 because of double differencing
        plt.axvline(x=elbow_point, color='red', linestyle='--', alpha=0.7, 
                   label=f'Suggested K = {elbow_point}')
        plt.legend()
    
    plt.show()
    
    return k_range, inertias

# Apply elbow method to synthetic data
print("Elbow Method Analysis for Synthetic Data:")
k_range_syn, inertias_syn = elbow_method(X_synthetic, max_k=10, title="Synthetic Dataset")

In [ ]:
# Apply elbow method to Iris data
print("Elbow Method Analysis for Iris Data:")
k_range_iris, inertias_iris = elbow_method(X_iris, max_k=8, title="Iris Dataset")

## 6. Silhouette Analysis for K Selection

### What is Silhouette Analysis?
Silhouette analysis measures how similar a point is to its own cluster compared to other clusters.

### Silhouette Score:
- **Range**: -1 to +1
- **+1**: Perfect clustering
- **0**: Point is on the border between clusters
- **-1**: Point might be assigned to wrong cluster

In [ ]:
def silhouette_analysis(X, max_k=10, title="Silhouette Analysis"):
    """Perform silhouette analysis to find optimal K"""
    k_range = range(2, max_k + 1)  # Silhouette score needs at least 2 clusters
    silhouette_scores = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(X)
        silhouette_avg = silhouette_score(X, cluster_labels)
        silhouette_scores.append(silhouette_avg)
    
    # Plot silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Average Silhouette Score')
    plt.title(f'{title} - Silhouette Analysis')
    plt.grid(True, alpha=0.3)
    
    # Mark the best K
    best_k = k_range[np.argmax(silhouette_scores)]
    best_score = max(silhouette_scores)
    plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, 
               label=f'Best K = {best_k} (Score: {best_score:.3f})')
    plt.legend()
    plt.show()
    
    return k_range, silhouette_scores, best_k

# Apply silhouette analysis to synthetic data
print("Silhouette Analysis for Synthetic Data:")
k_range_sil_syn, sil_scores_syn, best_k_syn = silhouette_analysis(X_synthetic, max_k=10, 
                                                                  title="Synthetic Dataset")

In [ ]:
# Apply silhouette analysis to Iris data
print("Silhouette Analysis for Iris Data:")
k_range_sil_iris, sil_scores_iris, best_k_iris = silhouette_analysis(X_iris, max_k=8, 
                                                                     title="Iris Dataset")

## 8. Apply K-Means with Optimal K

In [ ]:
# Summary of optimal K values from different methods
print("=== OPTIMAL K SELECTION SUMMARY ===\n")

print("Synthetic Dataset:")
print(f"True number of clusters: 4")
print(f"Silhouette Analysis suggests K = {best_k_syn}")
print("Elbow Method: Look at the plot above for the 'elbow' point\n")

print("Iris Dataset:")
print(f"True number of species: 3")
print(f"Silhouette Analysis suggests K = {best_k_iris}")

In [ ]:
# Apply K-Means with the optimal K values
def final_clustering_analysis(X, optimal_k, true_labels, title):
    """Perform final clustering with optimal K and compare with ground truth"""
    # Apply K-Means with optimal K
    kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    predicted_labels = kmeans_optimal.fit_predict(X)
    centroids = kmeans_optimal.cluster_centers_
    
    # Calculate metrics
    inertia = kmeans_optimal.inertia_
    silhouette_avg = silhouette_score(X, predicted_labels)
    
    # Create visualization comparing true vs predicted clusters
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # True clusters
    scatter1 = axes[0].scatter(X[:, 0], X[:, 1], c=true_labels, cmap='viridis', alpha=0.7)
    axes[0].set_title(f'{title} - True Clusters')
    axes[0].set_xlabel('Feature 1')
    axes[0].set_ylabel('Feature 2')
    
    # Predicted clusters
    scatter2 = axes[1].scatter(X[:, 0], X[:, 1], c=predicted_labels, cmap='viridis', alpha=0.7)
    axes[1].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=200, linewidths=3)
    axes[1].set_title(f'{title} - K-Means (K={optimal_k})\nSilhouette Score: {silhouette_avg:.3f}')
    axes[1].set_xlabel('Feature 1')
    axes[1].set_ylabel('Feature 2')
    
    plt.tight_layout()
    plt.show()
    
    return kmeans_optimal, predicted_labels

# Final clustering for synthetic data
print("Final Clustering Results:\n")
kmeans_final_syn, pred_syn = final_clustering_analysis(X_synthetic, best_k_syn, y_true, 
                                                       "Synthetic Dataset")

In [ ]:
# Final clustering for Iris data
kmeans_final_iris, pred_iris = final_clustering_analysis(X_iris, best_k_iris, y_iris_true, 
                                                         "Iris Dataset")

## 9. Comparison of K Selection Methods

In [ ]:
# Create a comprehensive comparison plot
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Elbow Method - Synthetic
axes[0, 0].plot(k_range_syn, inertias_syn, 'bo-', linewidth=2, markersize=6)
axes[0, 0].set_title('Elbow Method - Synthetic Data')
axes[0, 0].set_xlabel('Number of Clusters (K)')
axes[0, 0].set_ylabel('WCSS')
axes[0, 0].grid(True, alpha=0.3)

# Silhouette Analysis - Synthetic
axes[0, 1].plot(k_range_sil_syn, sil_scores_syn, 'go-', linewidth=2, markersize=6)
axes[0, 1].axvline(x=best_k_syn, color='red', linestyle='--', alpha=0.7)
axes[0, 1].set_title('Silhouette Analysis - Synthetic Data')
axes[0, 1].set_xlabel('Number of Clusters (K)')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].grid(True, alpha=0.3)

# Elbow Method - Iris
axes[1, 0].plot(k_range_iris, inertias_iris, 'bo-', linewidth=2, markersize=6)
axes[1, 0].set_title('Elbow Method - Iris Data')
axes[1, 0].set_xlabel('Number of Clusters (K)')
axes[1, 0].set_ylabel('WCSS')
axes[1, 0].grid(True, alpha=0.3)

# Silhouette Analysis - Iris
axes[1, 1].plot(k_range_sil_iris, sil_scores_iris, 'go-', linewidth=2, markersize=6)
axes[1, 1].axvline(x=best_k_iris, color='red', linestyle='--', alpha=0.7)
axes[1, 1].set_title('Silhouette Analysis - Iris Data')
axes[1, 1].set_xlabel('Number of Clusters (K)')
axes[1, 1].set_ylabel('Silhouette Score')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Key Takeaways and Best Practices

### Method Comparison:

1. **Elbow Method**:
   - **Pros**: Simple to understand and implement
   - **Cons**: "Elbow" point can be subjective
   - **Best for**: Quick initial assessment

2. **Silhouette Analysis**:
   - **Pros**: Considers both cohesion and separation
   - **Cons**: Can be computationally expensive
   - **Best for**: Balanced cluster evaluation

### Best Practices:
- Use multiple methods and compare results
- Consider domain knowledge about expected clusters
- Standardize features if they have different scales
- Validate results with business/domain expertise
- Consider the interpretability of the resulting clusters

### When to Use K-Means:
- ✅ Spherical clusters of similar size
- ✅ Clear separation between clusters
- ✅ Numerical features
- ❌ Non-spherical clusters
- ❌ Clusters of very different sizes
- ❌ High-dimensional data without preprocessing

In [ ]:
# Final summary
print("=== SESSION 10 SUMMARY ===\n")
print("📚 What we learned:")
print("1. Unsupervised learning finds patterns without labeled data")
print("2. K-Means clustering groups similar data points")
print("3. Multiple methods exist to determine optimal K:")
print("   - Elbow Method (WCSS analysis)")
print("   - Silhouette Analysis (cluster quality)")
print("4. Each method has strengths and limitations")
print("5. Combining multiple methods gives more robust results")

print("\n🎯 Results on our datasets:")
print(f"Synthetic Data (True K=4): Silhouette method suggested K={best_k_syn}")
print(f"Iris Data (True K=3): Silhouette method suggested K={best_k_iris}")

print("\n🚀 Next steps:")
print("- Try K-Means on your own datasets")
print("- Experiment with other clustering algorithms (Hierarchical, DBSCAN)")
print("- Learn about dimensionality reduction (PCA, t-SNE)")
print("- Explore clustering evaluation metrics")